In [ ]:
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd

import vice

from astropy.table import Table
import astropy as ap
import astropy.units as u

import surp

In [ ]:
surp.yields.set_magg22_scale()

In [ ]:
filename = "../../data/allStar-dr17-synspec_rev1.fits"



In [ ]:
ff = fits.open(filename)

In [ ]:
elem_props = Table.read(filename, format="fits", hdu=3, memmap=True)

In [ ]:
elem_props

In [ ]:
for i in range(27):
    print(elem_props["ELEM_SYMBOL"][i], "\t", elem_props["DWARF_SOLARNEIGH_ZERO"][i], " +- ", elem_props["DWARF_SOLARNEIGH_ZERO_ERR"][i], "\t", elem_props["SOLAR_ZERO"][i])

In [ ]:
for i in range(27):
    print(elem_props["ELEM_SYMBOL"][i], "\t", elem_props["GIANT_SOLARNEIGH_ZERO"][i], " +- ", elem_props["GIANT_SOLARNEIGH_ZERO_ERR"][i], "\t", elem_props["SOLAR_ZERO"][i])

In [ ]:
allStar = Table.read(filename, format="fits", hdu=1, memmap=True)

In [ ]:
filename = "../../data/apogee_astroNN-DR17.fits"
astroNN = Table.read(filename, format="fits", memmap=True)

In [ ]:
ann = astroNN.to_pandas()

In [ ]:
ann.drop_duplicates("APOGEE_ID", inplace=True)

In [ ]:
ann.set_index("APOGEE_ID", inplace=True)

remove stars with
- STAR_BAD
- NO_ASPCAP_RESULT
- EXTRATARG=0

- R=3-13kpc
- |Z| < 2kpc
- -0.75 < Mg/H < 0.45
- S/N > 200 and Mg/H > -0.5
- SN ? 100 otherwise
- log g = 1-2.5
- Teff = 4000-4600

In [ ]:
apogee_aspcapflag = 1<<23 # starbad
apogee_aspcapflag |= 1<<31 # no_aspcap_result

In [ ]:
extratarg = 1<<0

In [ ]:
extratarg

In [ ]:
mask_extratarg_weak = (allStar["EXTRATARG"] & extratarg)  == 0


In [ ]:
apogee_aspcapflag

In [ ]:
np.unique(allStar["ASPCAPFLAG"] & apogee_aspcapflag)

In [ ]:
mask_aspcap = (allStar["ASPCAPFLAG"] & apogee_aspcapflag) == 0
mask_extratarg = allStar["EXTRATARG"]  == 0


mask_fe_mg = allStar["FE_H_FLAG"] == 0
mask_fe_mg &= allStar["MG_FE_FLAG"] == 0

mask_fe_mg_2 = ~np.isnan(allStar["FE_H"])
mask_fe_mg_2 = ~np.isnan(allStar["MG_FE"])


In [ ]:
np.sum(mask_fe_mg), np.sum(mask_fe_mg_2), np.sum(mask_fe_mg_2 & mask_fe_mg)

In [ ]:
np.sum(mask_fe_mg_2 & mask_fe_mg)

In [ ]:
ann_filt = np.isin(allStar["APOGEE_ID"], astroNN["APOGEE_ID"])

In [ ]:
ann_match = ann.loc[allStar["APOGEE_ID"][ann_filt].value]

In [ ]:
ann_match

In [ ]:
r = np.full(len(allStar), np.NaN)
r[ann_filt] = ann_match.galr


z = np.full(len(allStar), np.NaN)
z[ann_filt] = ann_match.galz

In [ ]:
allStar["R_gal"] = r
allStar["z_gal"] = z

In [ ]:
plt.hist(allStar["R_gal"], range=(0, 10))

In [ ]:
plt.hist(allStar["z_gal"], range=(-5, 5))

In [ ]:
mask_zr = np.abs(allStar["z_gal"]) <= 2
mask_zr &= allStar["R_gal"] >= 3
mask_zr &= allStar["R_gal"] <= 13
mask_zr |= np.isnan(allStar["R_gal"])
mask_zr |= np.isnan(allStar["z_gal"])

print(np.sum(mask_zr))


In [ ]:
print(np.sum(mask_zr &  mask_extratarg))
print(np.sum(mask_zr & mask_extratarg_weak) )
print(np.sum(mask_zr  & mask_fe_mg & mask_aspcap & mask_fe_mg_2 ))

In [ ]:
mask = mask_zr  & mask_fe_mg & mask_aspcap & mask_fe_mg_2 & mask_extratarg
print(np.sum(mask))

In [ ]:
mg_h = allStar["MG_FE"] + allStar["FE_H"]

mask_mh = -0.75 <= mg_h
mask_mh &= mg_h <= 0.45

np.sum(mask_mh)
mask &= mask_mh
np.sum(mask)

In [ ]:
mask_snr = allStar["SNR"] >= 100
mask_snr &= (mg_h < -0.5)  | (allStar["SNR"] >= 200)

mask &= mask_snr
np.sum(mask)

In [ ]:
mask_logg = allStar["LOGG"] >= 1
mask_logg &= allStar["LOGG"] <= 2.5
np.sum(mask_logg)

mask &= mask_logg
np.sum(mask)

In [ ]:
mask_teff = allStar["TEFF"] <= 4600
mask_teff &= allStar["TEFF"] >= 4000

mask &= mask_teff
np.sum(mask)

In [ ]:
filtered = allStar[mask]

In [ ]:
len(filtered)

APOGEE CORRECTIONS
$$
{[{\rm{X}}/{\rm{H}}]}_{\mathrm{corr}}={[{\rm{X}}/{\rm{H}}]}_{\mathrm{APO}}+\mathrm{Offset}+{\alpha }_{T}({T}_{\mathrm{eff}}-4300)/100.
$$

Table 1 
Zero-point Offsets and T _eff Trend Slopes


| Elem.  |   Offset  |   10^3alpha_T  |
| -----  |    -----  |    ---------   |
| Mg     | 0.000     | 0.94           |
| O      | -0.016    |  2.28          |
| Si     | 0.038     | -3.22          |
| S      | 0.008     | 5.29           |
| Ca     | 0.071     | -6.01          |
| C+N    | 0.022     | 4.12           |
| Na     | 0.043     | 8.89           |
| Al     | 0.050     | -12.3          |
| K      | 0.002     | 1.68           |
| Cr     | 0.048     | 4.35           |
| Fe     | 0.053     | 0.76           |
| Ni     | 0.030     | 1.33           |
| V      | 0.222     | 14.9           |
| Mn     | 0.002     | 16.3           |
| Co     | -0.032    |  8.86          |
| Ce     | 0.125     | -2.64          |


In [ ]:
def correct(df, col, offset, alpha3):
    new = df[col] + offset + alpha3*1e-3*(df["TEFF"] - 3400)/100
    
    print(np.mean(new - df[col]))
    df[col] = new

In [ ]:
names = [name for name in filtered.colnames if len(filtered[name].shape) <= 1]


df1 = filtered[names].to_pandas()

df1["C_H"] = df1["C_FE"] + df1["FE_H"]
df1["N_H"] = df1["N_FE"] + df1["FE_H"]
df1["MG_H"] = df1["MG_FE"] + df1["FE_H"]

df1["CPN_H"] =  surp.gce_math.cpn(df1.C_H, df1.N_H)

correct(df1, "MG_H", 0, 0.94)
correct(df1, "FE_H", 0.053, 0.76)
correct(df1, "CPN_H", 0.022, 4.12)

In [ ]:


df1["MG_FE"] = df1.MG_H - df1.FE_H
df1["C_MG"] = df1["C_H"] - df1["MG_H"]
df1["N_MG"] = df1["N_H"] - df1["MG_H"]


df1["high_alpha"] = surp.gce_math.is_high_alpha(df1["MG_FE"], df1["FE_H"])

df1.to_csv("../../data/apogee_cartography.csv")

In [ ]:
df_lm = df1[(~df1.high_alpha) & (df1.MG_H < -0.3)]

In [ ]:
plt.hist2d(df1.FE_H, df1.MG_FE, 100);
plt.scatter(df_lm.FE_H, df_lm.MG_FE, s=0.3, alpha=0.1);

In [ ]:
plt.hist2d(df_lm.R_gal, df_lm.vt_gal,bins=100);
plt.axhline(220)

In [ ]:
plt.hist2d(df1.R_gal, df1.vt_gal,bins=100);
plt.axhline(220)

In [ ]:
plt.hist2d(df_lm.age, df_lm.vt_gal, bins=(np.linspace(0, 13, 100), 100));
plt.axhline(220)

In [ ]:
plt.hist2d(df1.age, df1.vt_gal, bins=(np.linspace(0, 13, 100), 100));
plt.axhline(220)

In [ ]:
len(df1)